In [ ]:
#IDCT

import numpy as np
import matplotlib.pyplot as plt

#コサイン定数の定義
C13 = -0.831469612
C9 = -0.195090322
C5 = 0.555570233

"""
C13 = -0.8
C9 = -0.2
C5 = 0.5
"""
#近い分数を調べてそれでやればいい？
#いや，intにするときの丸め誤差の方が大きい気がする
#それでもやらないとだめ
#丸め誤差があっても逆Cでずれなければ大丈夫な可能性

#課題：ハードウェアでやる場合の誤差伝搬をどう少なくするか

C5_inv = 1 / C5
C9_inv = 1 / C9
C13_inv = 1 / C13

print(C5_inv)
print(C9_inv)
print(C13_inv)

#8*8*8を生成(8*8の同じ画像を8枚)
#ランダム
"""
x_in1 = np.random.rand(8,8,8)*255
x_in1 = x_in1.astype(np.int64)
"""
#自作
x_in1 = np.copy(img_Cb)

X_out1 = np.zeros((8,8,8))
X_out2 = np.zeros((8,8,8))
X_out3 = np.zeros((8,8,8))
x_in3 = np.zeros((8,8,8))
x_in3i = np.zeros((8,8,8))
x_in2i = np.zeros((8,8,8))
x_in1i = np.zeros((8,8,8))
x_out1i = np.zeros((8,8,8))
x_out2i = np.zeros((8,8,8))

def butterfly_1(x):
    X = np.zeros(8)
    X[0] = x[0] + x[1]
    X[1] = x[2] + x[3]
    X[2] = x[0] - x[1]
    X[3] = x[2] - x[3]
    X[4] = x[4] + x[5]
    X[5] = x[6] + x[7]
    X[6] = x[4] - x[5]
    X[7] = x[6] - x[7]
    return X

def butterfly_2(x):
    X = np.zeros(8)
    X[0] = x[0] + x[1]
    X[1] = x[4] + x[5]
    X[2] = x[2] + x[3]
    X[3] = x[6] + x[7]
    X[4] = x[0] - x[1]
    X[5] = x[4] - x[5]
    X[6] = x[2] - x[3]
    X[7] = x[6] - x[7]
    return X

def butterfly_3(x):
    X = np.zeros(8)
    X[0] = x[0] + x[1]
    X[1] = x[0] - x[1]
    X[2] = x[2] + x[3]
    X[3] = x[2] - x[3]
    X[4] = x[4] + x[5]
    X[5] = x[4] - x[5]
    X[6] = x[6] + x[7]
    X[7] = x[6] - x[7]
    return X

def cosine(x):
    X = np.zeros(8)
    X[0] = x[0]
    X[1] = x[1]*C13
    X[2] = x[2]*C9
    X[3] = x[3]*C9*C13
    X[4] = x[4]*C5
    X[5] = x[5]*C13*C5
    X[6] = x[6]*C9*C5
    X[7] = x[7]*C13*C9*C5
    return X

def Ibutterfly_1(X):
    x = np.zeros(8)
    x[0] = 0.5*(X[0] + X[2])
    x[1] = 0.5*(X[0] - X[2])
    x[2] = 0.5*(X[1] + X[3])
    x[3] = 0.5*(X[1] - X[3])
    x[4] = 0.5*(X[4] + X[6])
    x[5] = 0.5*(X[4] - X[6])
    x[6] = 0.5*(X[5] + X[7])
    x[7] = 0.5*(X[5] - X[7])
    return x

def Ibutterfly_2(X):
    x = np.zeros(8)
    x[0] = (X[0] + X[4]) / 2.0
    x[1] = (X[0] - X[4]) / 2.0
    x[2] = (X[2] + X[6]) / 2.0
    x[3] = (X[2] - X[6]) / 2.0
    x[4] = (X[1] + X[5]) / 2.0
    x[5] = (X[1] - X[5]) / 2.0
    x[6] = (X[3] + X[7]) / 2.0
    x[7] = (X[3] - X[7]) / 2.0
    return x

def Ibutterfly_3(X):
    x = np.zeros(8)
    x[0] = 0.5 * (X[0] + X[1])
    x[1] = 0.5 * (X[0] - X[1])
    x[2] = 0.5 * (X[2] + X[3])
    x[3] = 0.5 * (X[2] - X[3])
    x[4] = 0.5 * (X[4] + X[5])
    x[5] = 0.5 * (X[4] - X[5])
    x[6] = 0.5 * (X[6] + X[7])
    x[7] = 0.5 * (X[6] - X[7])
    return x

def inv_cosine(X):
    x = np.zeros(8)
    x[0] = X[0]
    x[1] = X[1] * C13_inv
    x[2] = X[2] * C9_inv
    x[3] = X[3] * (C9_inv * C13_inv)
    x[4] = X[4] * C5_inv
    x[5] = X[5] * (C5_inv * C13_inv)
    x[6] = X[6] * (C5_inv * C9_inv)
    x[7] = X[7] * (C5_inv * C9_inv * C13_inv)
    return x

def DCT1(x_in):
    X_1 = butterfly_1(x_in)
    X_2 = butterfly_2(X_1)
    X_3 = butterfly_3(X_2)
    X_out = cosine(X_3)
    return X_out

def IDCT1(X_out):
    X_3 = inv_cosine(X_out)
    X_2 = Ibutterfly_3(X_3)
    X_1 = Ibutterfly_2(X_2)
    x = Ibutterfly_1(X_1)
    return x

def DCT(x_in):
    for i in range(8):
        x = x_in[i]
        X_out = DCT1(x)
        x_in[i] = X_out
    return x_in
    
def IDCT(X_out):
    for i in range(8):
        X = X_out[i]
        x = IDCT1(X)
        X_out[i] = x
    return X_out

def tenchi(x):
    for i in range(8):
        x[i] = np.transpose(x[i])
    return x


#print(x_in1)
x_before = np.zeros((8,8,8))
x_before = np.copy(x_in1)

#ＤＣＴを実行 
# ｘ方向   
for i in range(8):
    X_out1[i] = DCT(x_in1[i])
    
#転置
x_in2 = tenchi(X_out1)
#ｙ方向
for i in range(8):
    X_out2[i] = DCT(x_in2[i])
#ｚ方向
x_in3 = np.copy(X_out2)
x_in3 = np.rot90(x_in3, 1, axes=(2,0))
for i in range(8):
    X_out3[i] = DCT(x_in3[i])
#X_out3 = np.rot90(X_out3, -1, axes=(2,0)) #元に戻す回転（あえてやる必要はない）
X_out3 = X_out3.astype(np.int64)
print(X_out3)

#IDCTを実行
#z方向
for i in range(8):
    x_in3i[i] = IDCT(X_out3[i])
x_in3i = np.rot90(x_in3i, -1, axes=(2,0))
X_out2i = np.copy(x_in3i)
#y方向
for i in range(8):
    x_in2i[i] = IDCT(X_out2i[i])
#転置
X_out1i = tenchi(x_in2i)
# ｘ方向   
for i in range(8):
    x_in1i[i] = IDCT(X_out1i[i])

x_in1i = x_in1i.astype(np.int64)
#print(x_in1i)

x_dif = np.zeros((8,8,8))
x_dif = x_before - x_in1i
x_dif = x_dif.astype(np.int32)
print(x_before)
print(x_in1i)
print(x_dif)

#DCTの段階ごとにIDCTと比較する

#DCTも一つの関数にする

#量子化テーブルを，DCTの結果とJPGをもとに作成する
#圧縮・通信方式を考える

#画像の表示方法